<a href="https://colab.research.google.com/github/marzinouri/persianQA/blob/main/parsbert-finetuned-persianQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install transformers

In [3]:
%%capture
!pip install datasets

In [4]:
import transformers
model_checkpoint = "HooshvareLab/bert-fa-zwnj-base"
batch_size = 1

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset
datasets = load_dataset("SajjadAyoubi/persian_qa")

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,1328,کره جنوبی,کرهٔ جنوبی که با نام رسمی جمهوری کره (به کره‌ای: 대한민국) نیز شناخته می‌شود، کشوری است در بخش جنوبی شبه‌جزیرهٔ کره در شرق آسیا. پایتخت آن شهر سئول است. سئول پایتخت کره جنوبی و یک شهر نوین و صنعتی است. این شهر در سال ۱۹۴۸ م، پایتخت کشور شد. جمعیت این شهر بالغ بر ۱۰ میلیون نفر و بزرگترین شهر کشور کره جنوبی است. این کلان‌شهر یکی از بزرگترین شهرهای دنیا به حساب می‌آید. این کشور، با قوانینی که برابری اقتصادی و فرصت‌های برابر برای تمام مردم را تضمین می‌کند، میزبان شرکت‌های بزرگ و بین‌المللی است که اقتصاد آن را توانمند می‌سازند. تنش پیوسته با کره شمالی بانی آن شده‌است که کره جنوبی ۲٫۶٪ از تولید ناخالص داخلی کشور را به نگهداری و بهبود ارتش خود اختصاص دهد و ارتش این کشور، در رده‌های برتر قدرت نظامی جهان جای دارد.تاریخ کره از پارینه‌سنگی زیرین آغاز می‌شود و با چوسان و دوران باستان نمایان می‌شود؛ جایی که گوریو و امپراتوری گوگوریو شکل گرفتند. در حدود ۱۹۷۹ میلادی، این کشور درگیر تنش‌های سیاسی و اقتصادی فراوانی بود و پس از این سال، تغییرات کم‌کم پدید آمدند,چه چیزی باعث شده ارتش کره شمالی قدرتمند باشد؟,"{'text': [], 'answer_start': []}"
1,6130,دریا,دریا بدنه آبی متصل آب‌های شور می‌باشد که ۷۰٫۸ درصد سطح زمین را پوشش می‌دهد. دریا آب‌وهوای زمین را معتدل می‌کند و نقش مهمی در چرخه آب، چرخه کربن، و چرخه نیتروژن دارد. اگرچه از زمان پیشاتاریخ دریانوردی و اکتشاف در دریا رواج داشته‌است، مطالعه مدرن علمی دریا (اقیانوس‌نگاری) به‌طور گسترده به سفر چلنجر بریتانیا در دهه ۱۸۷۰ مربوط می‌شود. دریا به‌طور مرسوم به چهار یا پنج بخش بزرگ تقسیم شده‌است، مواردی مانند آرام را اقیانوس و مواردی مانند مدیترانه را دریا می گویند.«واژه زرنگ کهن‌ترین نام سیستان و زاولستان است و در سنگ‌نوشتهٔ بیستون به گونهٔ زرنگا آمده‌است. به باور پژوهشگران، زرنگ و زریه که به زبان اوستایی به معنای دریا است، و دریه به فارسی هخامنشی و زریا در زبان پهلوی و دریا به زبان امروزی همه یکی است.» بزرگ‌ترین دریاهای جهان عبارتند از: دریای مدیترانه، دریای برینگ، دریای کارائیب، دریای اختسک. با توجه به شرایط فعلی رانش قاره‌ای، نیم‌کره شمالی اکنون به‌طور نسبی بین خشکی و دریا تقسیم شده‌است (با نسبت ۲:۳) اما نیم‌کره جنوبی به شدت اقیانوسی است (1:4.7).,هخامنشیان در چه تاریخی زندگی می کردند؟,"{'text': [], 'answer_start': []}"
2,3051,بلغارستان,بلغارستان (به بلغاری: България) با نام رسمی جمهوری بلغارستان (به بلغاری: Република България) کشوری در جنوب شرقی اروپا است. پایتخت این کشور شهر صوفیه است. جمعیت این کشور بالغ بر ۷٬۳۶۴٬۵۷۰ (در سال 2020) نفر است که هر ساله ۱٪ از آن کاسته می‌شود. بلغارستان بیشترین رشد منفی جمعیت را در جهان دارد.زبان رسمی این کشور بلغاری و واحد پول آن لِف با واحد جز ستونیکیی است. حدود ۸۵ درصد از مردم آن بلغاری‌تبار و بقیه بیشتر ترک و کولی هستند. بلغارستان ۱۱۰٬۹۹۴ کیلومتر مربع مساحت دارد و چهاردهمین کشور بزرگ در اروپا است. بلغارستان در سال ۲۰۰۴ به عضویت سازمان پیمان آتلانتیک شمالی (ناتو) درآمد و در سال ۲۰۰۵ اتحادیه اروپا، درخواست عضویت بلغارستان را در این اتحادیه پذیرفت و روز عضویت این کشور در اتحادیه اروپا را یک ژانویه ۲۰۰۷ اعلام کرد. مهمترین صادرات بلغارستان اسانس گل رز است، همچنین شراب نیز از دیگر اقلام مهم صادراتی بلغارستان است، بلغارستان چهارمین صادرکننده شراب در جهان است، پوشاک، کفش، آهن و فولاد، ماشین آلات و تجهیزات و سوخت نیز از دیگر صادرات بلغارستان می‌باشد. جمهوری بلغارستان رسماً کشوری سکولار است اما در قانون اساسی آن مسیحیت ارتدکس به عنوان دین سنتی کشور ذکر شده‌است.۷۶٪ جمعیت پیرو آیین مسیحیت نوع ارتدوکس هستند و ۱۰٪ جمعیت را مسلمانان تشکیل می‌دهند و ۱۱٫۸ درصد بی‌دین هستند.,بلغارستان در کدام قسمت اروپا قرار دارد؟,"{'text': ['جنوب شرقی'], 'answer_start': [102]}"
3,5894,چاقی,چاقی وضیعت پزشکی است که بافت چربی بیش از حد طبیعی در بدن فرد انباشته شده باشد. انباشت بیش از حد بافت چربی می‌تواند باعث پسرفت شاخص‌های سلامتی، از جمله کاهش میانگین طول عمر و/یا کاهش کیفیت زندگی گردد. شایع‌ترین روش برای تخمین چاقی استفاده از شاخص توده بدن (BMI) است. شاخصی که با تقسیم کردن وزن شخص بر حسب کیلوگرم بر توان دوم قد وی بر حسب متر به دست می‌آید. طبق تعریف هنگامی که شاخص توده بدن از ۳۰=(kg/m^۲) بیشتر شود، آن فرد چاق محسوب می‌شود. و در کودکان از صدک استفاده میشود دلایل متنوعی برای علت چاقی ارائه شده‌است. اما مهم‌ترین علت مؤثر در چاقی و اضافه وزن، عبارت است از تما

In [11]:
show_random_elements(datasets["validation"])

,id,title,context,question,answers
0,9387,ریچارد یکم,ریچارد در سومین جنگ صلیبی شرکتی مؤثر داشت و یکی از سرداران صلیبیون علیه صلاح‌الدین ایوبی بود که به شکست انجامید و خزانه به جا مانده از هنری دوم را به کلی تهی کرد. بعداً در هنگام بازگشت به وطن در آلمان اسیر لئوپولد پنجم، دوک اتریش گردید و برای آزادی وی بیش از دو برابر درآمد سالانه خزانه‌داری انگلستان درخواست شد. در جریان جنگ صلیبی سوم سه هزار نفر از اسرای مسلمان را در پای دیوارهای عکا و پیش چشم صلاح الدین ایوبی قتل‌عام کرد. پس از آزادی به جنگی پرهزینه علیه پادشاه فرانسه فیلیپ دوم معروف به فیلیپ اوگوست (ستایششده) دست زد که حاصلی در پی نداشت. سرانجام این پادشاه ولخرج و ماجراجو در جریان محاصره قلعه‌ای در فرانسه در اثر اصابت تیری زهرآلود کشته شد. افسانه رابین‌هود و راهزنان جنگل شروود در زمان یکی از سفرهای ریچارد اتفاق می‌افتد که حکومت در اختیار برادر ریچارد یعنی جان ملقب به بی‌زمین (لکلند) بوده‌است. در شرف پایان سال ۱۱۸۹ میلادی، هنوز هنری دوم و فیلیپ دوم خود را آماده حرکت به سرزمین مقدس نکرده بودند. در این زمان ریچارد، پسر هنری دریافت که پدرش درصدد انتخاب برادرش، جان لکلند، به عنوان جانشین خود است؛ بنابراین با فیلیپ دوم متحد و وارد جنگ با پدرش شد.,ریچارد چندتا داداش داشت؟,"{'text': [], 'answer_start': []}"
1,9842,ژن,"ژن، در ژنتیک مندلی و ژنتیک مولکولی کلاسیک, همچنین در مبحث تکامل و در دوران پساژنگان تعاریف متفاوتی دارد اما از دیدگاه مولکولی کلاسیک دنباله‌ای از نوکلوئوتیدهاست که در برگیرنده اطلاعات لازم جهت تولید مولوکول‌های RNA و یا پروتئین‌های لازم برای سلول هستند. هر ژن در بخشی از DNA سلول وجود دارد.یا با تعریفی دیگر ژن بخشی از مولکول دی ان ای می باشد که روی یک رشته از آن قرار دارد. درون سلول‌ها، طی فرایند رونویسی ژن‌ها به مولکول‌های RNA تبدیل می‌شوند که یا به شکل مستقیم در سلول استفاده می‌شوند و یا دربرگیرنده اطلاعاتی جهت تولید پروتئین هستند و طی فرایند ترجمه، پروتئین مربوط به آن‌ها ساخته می‌شود. ژن‌ها می‌توانند دچار جهش شوند، که تغییری در توالی دنباله آن‌هاست. جهش‌ها باعث ایجاد تغییر در پروتئین‌های تولید شده از روی ژن‌ها می‌شوند و می‌توانند عملکرد پروتئین را به کلی تغییر دهند و باعث به وجود آمدن صفاتی جدید در سلول بشوند. ژن‌ها در اثر جهش‌های ژنتیکی، براساس انتخاب طبیعی تکامل پیدا می‌کنند و ژن‌های قوی‌تر و بهتر به مرور جایگزین ژن‌های قبلی می‌شوند.",ژن چگونه به RNA تبدیل می شود؟,"{'text': ['طی فرایند رونویسی', 'رونویسی'], 'answer_start': [389, 399]}"
2,10102,مارک زاکربرگ,مارک الیوت زاکربرگ برنامه‌نویس رایانه و یکی از مؤسسان وبگاه شبکه اجتماعی آنلاین فیس‌بوک در آمریکا است. مجلهٔ تایم وی را به عنوان شخصیت سال ۲۰۱۰ انتخاب کرده‌است. او به عنوان یک دانشجوی دانشگاه هاروارد با کمک دانشجویان علوم کامپیوتر به خصوص پشتوانهٔ «اندرو مک‌کالم»، «داستین موسکوویتز» و «کریس هاگز» اقدام به تأسیس این وبگاه کرد. او اکنون به عنوان مدیر اجرایی در فیس‌بوک فعالیت می‌کند. در سال ۲۰۰۸، مجله فوربس او را جوانترین میلیاردر، که ثروتش را شخصاً کسب کرده معرفی کرد و ثروت او را ۱٬۵ میلیارد دلار تخمین زده‌است. هم‌اکنون وبگاه فوربس، دارایی‌های وی را ۷۸٬۳ میلیارد دلار اعلام کرده‌است و از این لحاظ پنجمین شخص ثروتمند در جهان است. در سال ۲۰۱۰ فیلمی با عنوان شبکهٔ اجتماعی به کارگردانی دیوید فینچر در مورد مارک زاکربرگ و فیس‌بوک ساخته شد. این فیلم که براساس کتاب میلیاردرهای تصادفی نوشتهٔ بن مزریک تهیه شده، به نحوهٔ شکل‌گیری و تأسیس فیس‌بوک و مسائل و کشمکش‌های پیرامون آن می‌پردازد. قابل ذکر است که نه مارک زاکربرگ و نه هیچ‌کدام از شرکای نزدیک وی در تهیه و ساخت این فیلم همکاری نداشته‌اند.,ثروتمند ترین فرد دنیا کیست؟,"{'text': [], 'answer_start': []}"
3,9653,باشگاه فوتبال آرسنال,باشگاه فوتبال آرسنال یک باشگاه فوتبال انگلیسی در شمال شهر لندن است که موفق به کسب ۱۳ عنوان قهرمانی در لیگ برتر انگلستان، ۱۴ قهرمانی در جام حذفی فوتبال انگلستان ، ۱۶ قهرمانی در جام خیریه انگلستان و دو قهرمانی در جام اتحادیه فوتبال انگلستان شده‌است. آن‌ها رکورددار طولانی‌ترین مدت صدرنشینی بدون وقفه در لیگ فوتبال انگلیس، بیشترین بازی بدون باختِ پیاپی (۴۹ بازی) و همچنین قهرمانی بدون شکست در یک فصل (۰۴–۲۰۰۳) می‌باشند و توانستند اولین و تنها تیمی در تاریخ لیگ برتر باشند که جام طلایی را بدست می‌آورند. باشگاه فوتبال آرسنال در سال ۱۸۸۶ در محله وول‌ویچ واقع در جنوب‌شرقی لندن شکل گرفت و در سال ۱۸۹۳ میلادی برای اولین بار از جنوب انگلستان به فوتبال لیگ پیوست. در

In [8]:
max_length = 384
doc_stride = 128

In [9]:
pad_on_right = tokenizer.padding_side == "right"


In [10]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:

                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [14]:
# features = prepare_train_features(datasets['train'][:5])


In [11]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)


Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

In [12]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [14]:
torch.cuda.is_available()

True

In [20]:
%%capture
!pip install accelerate -U

In [15]:
model_name = "parsbert-finetuned-persianQA"
args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    save_steps=5000
)

In [16]:
from transformers import default_data_collator

data_collator = default_data_collator

In [19]:
trainer = Trainer(
    model.to(device),
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [20]:
#my system ran out of memory after 2 epochs. I continued training from a saved checkpoint

trainer.train()

Epoch,Training Loss,Validation Loss
1,2.988100,2.565369
2,2.278700,2.563670


Epoch,Training Loss,Validation Loss
1,2.988100,2.565369
2,2.278700,2.563670
3,1.350400,3.371840


TrainOutput(global_step=27615, training_loss=2.4094787166663836, metrics={'train_runtime': 3517.9325, 'train_samples_per_second': 7.85, 'train_steps_per_second': 7.85, 'total_flos': 5411782452948480.0, 'train_loss': 2.4094787166663836, 'epoch': 3.0})

In [ ]:
trainer.train("C:/Users/marzi/Downloads/parsbert-finetuned-persianQA/checkpoint-20000")

Loading model from C:/Users/marzi/Downloads/parsbert-finetuned-persianQA/checkpoint-20000).
***** Running training *****
  Num examples = 9205
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 27615
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 20000
  Will skip the first 2 epochs then the first 1590 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/1590 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
3,1.470300,3.428255


Saving model checkpoint to parsbert-finetuned-persianQA\checkpoint-25000
Configuration saved in parsbert-finetuned-persianQA\checkpoint-25000\config.json
Model weights saved in parsbert-finetuned-persianQA\checkpoint-25000\pytorch_model.bin
tokenizer config file saved in parsbert-finetuned-persianQA\checkpoint-25000\tokenizer_config.json
Special tokens file saved in parsbert-finetuned-persianQA\checkpoint-25000\special_tokens_map.json
tokenizer config file saved in parsbert-finetuned-persianQA\tokenizer_config.json
Special tokens file saved in parsbert-finetuned-persianQA\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 930
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=27615, training_loss=0.3851581684687896, metrics={'train_runtime': 7296.5564, 'train_samples_per_second': 3.785, 'train_steps_per_second': 3.785, 'total_flos': 5411782452948480.0, 'train_loss': 0.3851581684687896, 'epoch': 3.0})

In [21]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [22]:
output.start_logits.shape, output.end_logits.shape


(torch.Size([1, 384]), torch.Size([1, 384]))

In [23]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)


(tensor([19], device='cuda:0'), tensor([19], device='cuda:0'))

In [24]:
n_best_size = 20


In [25]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""
                }
            )

In [26]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [27]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

In [28]:
raw_predictions = trainer.predict(validation_features)


In [29]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
max_answer_length = 30


In [30]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
context = datasets["validation"][0]["context"]
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index:
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 17.286743, 'text': 'مادرید'},
 {'score': 14.515724, 'text': 'در مادرید'},
 {'score': 4.037961, 'text': 'مادرید،'},
 {'score': 3.554978,
  'text': 'مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 2.792417,
  'text': 'رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 2.3897376,
  'text': 'یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 1.8212633, 'text': 'فوتبال است که در مادرید'},
 {'score': 1.5499735, 'text': 'مادرید، پایتخت اسپانیا'},
 {'score': 1.5446272,
  'text': 'باشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 1.2669418, 'text': 'در مادرید،'},
 {'score': 0.8022642, 'text': 'مادرید، پایتخت اسپانیا قرار دارد'},
 {'score': 0.34256458, 'text': 'باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 0.29788113, 'text': 'مادرید، پایتخت اسپانیا قرار دارد.'},
 {'score': 0.28706264, 'text': 'حرفه\u200cای فوتبال است که در مادرید'},
 {'score': -0.3963647, 'text': 'مادرید، پایتخت اسپان

In [31]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [32]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()


    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []

        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [33]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)


Post-processing 930 example predictions split into 930 features.


  0%|          | 0/930 [00:00<?, ?it/s]

In [35]:
from datasets import load_metric
metric = load_metric("squad_v2")
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

In [36]:
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 41.935483870967744,
 'f1': 57.93013297514893,
 'total': 930,
 'HasAns_exact': 30.87557603686636,
 'HasAns_f1': 53.725074757125164,
 'HasAns_total': 651,
 'NoAns_exact': 67.74193548387096,
 'NoAns_f1': 67.74193548387096,
 'NoAns_total': 279,
 'best_exact': 42.1505376344086,
 'best_exact_thresh': 0.0,
 'best_f1': 58.03765985686937,
 'best_f1_thresh': 0.0}